# 1. record point cloud

In [6]:
import sys
import os

# Get the current working directory (used to replace __file__)
current_dir = os.getcwd()

target_path1 = os.path.abspath(os.path.join(current_dir, '../3_move'))
target_path2 = os.path.abspath(os.path.join(current_dir, '../1_getPointCloud'))

# Add the paths to sys.path
if target_path1 not in sys.path:
    sys.path.append(target_path1)
    
if target_path2 not in sys.path:
    sys.path.append(target_path2)
    
# Check if the paths were added successfully
print("Current Python Path:")
print("\n".join(sys.path))

from ImageRecognizer import ImageRecognizer
image_recognizer = ImageRecognizer(top_dir="/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/cubes/")

from utils import matrix_to_rpy_and_translation

from PickAndPlace import PickAndPlace
pick_place = PickAndPlace(approach_distance=0.2)

Current Python Path:

/opt/ros_ws/devel/lib/python3/dist-packages
/opt/ros/noetic/lib/python3/dist-packages
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload
/opt/ros_ws/irobmanenv/lib/python3.8/site-packages
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/3_move
/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/1_getPointCloud
/opt/ros_ws/irobmanenv/lib/python3.8/site-packages/setuptools/_vendor
[INFO] [1742596897.347842, 669.222000]: Waiting for move_group action server...
[INFO] [1742596897.605754, 669.226000]: Table added to the scene to prevent collision below z = 0.001
[ INFO] [1742596897.600263585, 669.458000000]: Ready to take commands for planning group panda_manipulator.
[INFO] [1742596897.615286, 669.466000]: Wall 'wall_right' added to the scene with rotation theta=-0.6283185307179586 radians at position: 0.0, 0.8, 0.0
[INFO] [1742596897.624432, 669.477000]: Wall 'wall_left' added to the scene with rotation theta=0.6283185307179586 radians a

In [7]:
from save_point_cloud import PointCloudSaver
import open3d as o3d
import rospy
point_cloud_saver = PointCloudSaver()

# Wait for data to be ready
rospy.loginfo("Waiting for data...")
rospy.sleep(1)  # Wait for topic data to be published

# Save the point cloud
world_file = "/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/mesh/zed_point_cloud_world3.ply"
point_cloud_saver.save_point_clouds(world_file)

[INFO] [1742596910.979215, 682.120000]: Waiting for data...
[INFO] [1742596911.196814, 682.299000]: Received image message.
[INFO] [1742596911.295750, 682.335000]: Received point cloud data.
[INFO] [1742596915.892342, 686.479000]: Color range - Min: [0.10196078 0.10196078 0.10196078], Max: [0.75294118 0.63921569 0.60784314]
[INFO] [1742596915.911566, 686.497000]: Requesting transform from world to left_camera_link_optical...
[INFO] [1742596915.918164, 686.497000]: Transform found: header: 
  seq: 0
  stamp: 
    secs: 686
    nsecs: 467000000
  frame_id: "world"
child_frame_id: "left_camera_link_optical"
transform: 
  translation: 
    x: 0.20991587728489502
    y: -0.05995457173018651
    z: 0.5609911284057665
  rotation: 
    x: 0.6590531542888034
    y: 0.6591330828003503
    z: 0.25611571331347577
    w: 0.2561196212202706
[INFO] [1742596916.662878, 687.157000]: Transformed point cloud saved to /opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/mesh/zed_point_cloud

In [8]:
import open3d as o3d
import numpy as np
from utils import filter_point_cloud_by_depth_and_range, filter_point_cloud_by_depth

# zed_ply_path = "mesh/zed_point_cloud_world3.ply"
zed_ply_path = "/opt/ros_ws/src/franka_zed_gazebo/scripts/mycode_clean/2_perception/mesh/zed_point_cloud_world3.ply"

coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # Size of the coordinate axes, can be adjusted as needed
    origin=[0, 0, 0]  # Origin of the coordinate axes
)

# Read the point cloud file
point_cloud = o3d.io.read_point_cloud(zed_ply_path)
if not point_cloud.has_points():
    raise ValueError(f"Failed to read point cloud from {zed_ply_path}")
filtered_point_cloud = filter_point_cloud_by_depth_and_range(point_cloud, depth_threshold=0.02, range=[0.001, -0.8, 1, 1.6])
o3d.visualization.draw_geometries([filtered_point_cloud, coordinate_frame], window_name="Filtered Point Cloud")

error: XDG_RUNTIME_DIR not set in the environment.


In [4]:
# from utils import calculate_max_layer

# # Example usage
# max_layer = calculate_max_layer(filtered_point_cloud, layer_height=0.04)
# print(f"MaxLayer: {max_layer}")

# 2. Perception

## 2.1 Coarse and Fine registration 

In [9]:
import copy
import open3d as o3d
import numpy as np

def register_and_filter(pointcloud, mesh, voxel_size=0.01):
    # Convert mesh to point cloud
    if isinstance(mesh, o3d.geometry.TriangleMesh):
        mesh_pointcloud = mesh.sample_points_uniformly(number_of_points=1000)
    elif isinstance(mesh, o3d.geometry.PointCloud):
        mesh_pointcloud = copy.deepcopy(mesh)
    
    # Downsample the point cloud and compute features
    def preprocess_point_cloud(pcd, voxel_size):
        pcd_down = pcd.voxel_down_sample(voxel_size)
        pcd_down.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=voxel_size*2, 
                max_nn=30
            )
        )
        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            pcd_down,
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=voxel_size*5, 
                max_nn=100
            )
        )
        return pcd_down, pcd_fpfh

    # Coarse registration
    def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
        distance_threshold = voxel_size * 1.5
        result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, True,
            distance_threshold,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            4,
            [
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ],
            o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
        )
        return result

    # Fine registration
    def refine_registration(source, target, initial_transformation, voxel_size):
        distance_threshold = voxel_size * 1  # Reduce the threshold for higher accuracy
        result = o3d.pipelines.registration.registration_icp(
            source, target, distance_threshold, initial_transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000000)
        )
        return result

    # Execute point cloud preprocessing
    source_down, source_fpfh = preprocess_point_cloud(mesh_pointcloud, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(pointcloud, voxel_size)

    # Execute registration
    coarse_result = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
    refined_result = refine_registration(mesh_pointcloud, pointcloud, coarse_result.transformation, voxel_size)

    # Transform the mesh point cloud
    transform = refined_result.transformation
    transformed_mesh_pointcloud = mesh_pointcloud.transform(transform)

    # Create bounding box
    oriented_bounding_box = transformed_mesh_pointcloud.get_oriented_bounding_box()
    center = oriented_bounding_box.center
    extent = oriented_bounding_box.extent
    rotation_matrix = oriented_bounding_box.R

    # Expand the bounding box
    margin = voxel_size
    expanded_extent = extent + 0.5 * margin
    expanded_bounding_box = o3d.geometry.OrientedBoundingBox(
        center=center,
        extent=expanded_extent,
        R=rotation_matrix
    )

    # Filter the point cloud
    indices_inside_box = expanded_bounding_box.get_point_indices_within_bounding_box(pointcloud.points)
    indices_outside_box = list(set(range(len(pointcloud.points))) - set(indices_inside_box))

    # Separate the point cloud
    remaining_pointcloud = pointcloud.select_by_index(indices_outside_box)
    deleted_pointcloud = pointcloud.select_by_index(indices_inside_box)

    return transform, remaining_pointcloud, deleted_pointcloud, refined_result.fitness


In [10]:
# Read the cube
# Define file paths
cube_obj_path = "mesh/cube_0.obj"
zed_ply_path = "mesh/zed_point_cloud_world3.ply"

# Read the cube_0.obj mesh
cube_mesh = o3d.io.read_triangle_mesh(cube_obj_path)
cube_mesh.compute_vertex_normals()  # Compute normals for better visualization
# cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000)  # Convert to point cloud
# cube_point_cloud = cube_mesh.sample_points_poisson_disk(number_of_points=1000)

# Remove the lower part of the cube to prevent flipping along the z-axis
# cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.015)
# o3d.visualization.draw_geometries([cube_point_cloud])


coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1,  # Size of the coordinate axes, can be adjusted as needed
    origin=[0, 0, 0]  # Origin of the coordinate axes
)


In [11]:
def check_transform_z_axis_alignment(transform, tolerance=0.1):
    """
    Check if the Z-axis of the transform is parallel to the Z-axis of the world coordinate system.
    Allows a certain tolerance range to check if it is parallel or anti-parallel.
    
    Args:
    - transform: 4x4 transformation matrix
    - tolerance: Tolerance range for checking, default is 0.1
    
    Returns:
    - True: If the Z-axis is parallel or anti-parallel
    - False: If the Z-axis is not parallel
    - The corrected transform
    """
    z_axis = np.array([0, 0, 1])  # Z-axis of the world coordinate system
    transform_z_axis = transform[:3, 2]  # Get the Z-axis of the transform (i.e., the third column of the rotation matrix)

    # Compute the angle between the transform's Z-axis and the world coordinate system's Z-axis
    dot_product = np.dot(transform_z_axis, z_axis)
    # Compute the cosine of the angle, if close to 1 or -1, it means parallel or anti-parallel
    if np.abs(dot_product) > (1 - tolerance):
        return True
    return False

def align_transform_z_axis(transform):
    """
    If the Z-axis of the transform is anti-parallel to the Z-axis of the world coordinate system,
    rotate by 180 degrees around the X-axis (np.pi) to flip the direction of the Z-axis.
    
    Args:
    - transform: 4x4 transformation matrix
    
    Returns:
    - The corrected transform matrix
    """
    z_axis_world = np.array([0, 0, 1])  # Z-axis of the world coordinate system
    transform_z_axis = transform[:3, 2]  # Get the Z-axis of the transform (the third column of the rotation matrix)

    # Check if the Z-axis is anti-parallel to the world coordinate system's Z-axis
    if np.dot(transform_z_axis, z_axis_world) < 0:  # Z-axis is anti-parallel
        print("Correcting Z axis")
        # Create a rotation matrix to rotate 180 degrees around the X-axis
        rotation_matrix = np.eye(4)
        rotation_matrix[1, 1] = -1  # Rotate the matrix by 180 degrees around the X-axis
        rotation_matrix[2, 2] = -1  # Rotate the matrix by 180 degrees around the X-axis
        
        # Perform matrix multiplication, applying the rotation matrix to the original transform
        transform = np.dot(rotation_matrix, transform)

    return transform

def align_transform_z_axis_any_orientation(transform):
    """
    Aligns the Z-axis of the transform with the negative Z-axis of the world coordinate system,
    no matter what the original orientation is.
    
    Args:
    - transform: 4x4 transformation matrix
    
    Returns:
    - The corrected transform matrix with Z-axis pointing downward
    """
    # Extract the original rotation matrix and translation vector
    rotation = transform[:3, :3]
    translation = transform[:3, 3]
    
    # Extract the current z-axis direction (third column of rotation matrix)
    current_z = rotation[:, 2]
    
    # Target z-axis direction (downward in world coordinates)
    target_z = np.array([0, 0, -1])
    
    # Check if already aligned within a small tolerance
    if np.allclose(current_z, target_z, atol=1e-6):
        return transform
    
    # For completely anti-aligned case (pointing directly up), simple 180° rotation works
    if np.allclose(current_z, -target_z, atol=1e-6):
        # Rotate 180 degrees around x-axis
        r = np.eye(3)
        r[1, 1] = -1
        r[2, 2] = -1
        new_rotation = rotation @ r
    else:
        # For any other orientation, we need to find the rotation that aligns vectors
        # Compute the rotation axis (cross product of current and target z-axes)
        rotation_axis = np.cross(current_z, target_z)
        
        # If current_z and target_z are parallel or anti-parallel, rotation_axis might be zero
        # In that case, choose any perpendicular axis
        if np.allclose(rotation_axis, 0, atol=1e-10):
            # Find a non-zero component in current_z
            if abs(current_z[0]) > 1e-10:
                rotation_axis = np.array([current_z[1], -current_z[0], 0])
            else:
                rotation_axis = np.array([0, current_z[2], -current_z[1]])
        
        # Normalize the rotation axis
        rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
        
        # Compute the rotation angle (dot product gives cosine of angle)
        # We want the shorter arc between the vectors
        cos_angle = np.dot(current_z, target_z)
        angle = np.arccos(np.clip(cos_angle, -1.0, 1.0))
        
        # Rodrigues' rotation formula to get rotation matrix
        K = np.array([
            [0, -rotation_axis[2], rotation_axis[1]],
            [rotation_axis[2], 0, -rotation_axis[0]],
            [-rotation_axis[1], rotation_axis[0], 0]
        ])
        R = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * (K @ K)
        
        # Apply the rotation to the original rotation matrix
        new_rotation = rotation @ R
    
    # Create the new transformation matrix
    new_transform = np.eye(4)
    new_transform[:3, :3] = new_rotation
    new_transform[:3, 3] = translation
    
    # Validate the result
    result_z = new_transform[:3, :3][:, 2]
    alignment_quality = np.dot(result_z, target_z)
    print(f"Z-axis alignment quality: {alignment_quality:.6f} (closer to 1 is better)")
    
    return new_transform

def align_transform_z_axis_preserving_xy(transform):
    """
    Align the Z-axis of the transformation matrix to point downward (world -Z axis),
    while preserving the original XY plane orientation as much as possible.

    This function ensures:
    1. The Z-axis points downward (aligned with the world -Z axis)
    2. The XY plane is preserved as much as possible to minimize disruption to the original grasp direction

    Args:
        transform: A 4x4 transformation matrix

    Returns:
        A corrected transformation matrix with the Z-axis pointing downward and XY directions as close as possible to the original
    """
    # Extract the original rotation matrix and translation vector
    rotation = transform[:3, :3]
    translation = transform[:3, 3]

    # Extract current axes directions from the rotation matrix
    x_axis = rotation[:, 0]  # X-axis is the first column
    y_axis = rotation[:, 1]  # Y-axis is the second column
    z_axis = rotation[:, 2]  # Z-axis is the third column

    # Target direction for Z-axis (world -Z direction)
    target_z = np.array([0, 0, -1])

    # Check if the current Z-axis is already aligned with the target
    aligned = np.isclose(np.abs(np.dot(z_axis, target_z)), 1.0, atol=1e-6)
    if aligned and np.dot(z_axis, target_z) > 0:
        # Already aligned, no changes needed
        return transform

    # Set new Z-axis to point down
    new_z = target_z

    # Compute a new X-axis that is orthogonal to the new Z-axis
    # Do this by projecting the original X-axis onto the plane perpendicular to new Z

    new_x = x_axis - np.dot(x_axis, new_z) * new_z

    # If the projection is too small (i.e., X was almost aligned with Z), try using original Y
    if np.linalg.norm(new_x) < 1e-6:
        new_x = y_axis - np.dot(y_axis, new_z) * new_z

    # If still too small, pick an arbitrary vector that's not parallel to Z
    if np.linalg.norm(new_x) < 1e-6:
        temp = np.array([1, 0, 0]) if abs(new_z[0]) < 0.9 else np.array([0, 1, 0])
        new_x = temp - np.dot(temp, new_z) * new_z

    # Normalize the new X-axis
    new_x = new_x / np.linalg.norm(new_x)

    # Calculate the new Y-axis using the cross product (ensures orthogonality)
    new_y = np.cross(new_z, new_x)

    # Construct the new rotation matrix from the orthonormal axes
    new_rotation = np.column_stack((new_x, new_y, new_z))

    # Assemble the final transformation matrix
    new_transform = np.eye(4)
    new_transform[:3, :3] = new_rotation
    new_transform[:3, 3] = translation

    # Evaluate alignment quality of the new Z-axis
    result_z = new_transform[:3, :3][:, 2]
    alignment_quality = np.dot(result_z, target_z)
    print(f"Z-axis alignment quality: {alignment_quality:.6f} (closer to 1 is better)")

    # Evaluate how well the XY plane orientation is preserved
    original_xy_normal = np.cross(x_axis, y_axis)
    original_xy_normal = original_xy_normal / np.linalg.norm(original_xy_normal)

    new_xy_normal = np.cross(new_x, new_y)
    new_xy_normal = new_xy_normal / np.linalg.norm(new_xy_normal)

    xy_preservation = np.abs(np.dot(original_xy_normal, new_xy_normal))
    print(f"XY plane preservation quality: {xy_preservation:.6f} (closer to 1 is better)")

    return new_transform


In [12]:
import rospy
import tf
import numpy as np
from geometry_msgs.msg import TransformStamped

class TransformBroadcaster:
    def __init__(self):
        # Try to initialize the ROS node, avoid initializing multiple times
        try:
            rospy.init_node('tf_broadcaster_node')
        except rospy.exceptions.ROSException:
            pass  # If the node is already initialized, do nothing

        # Create a TransformBroadcaster instance
        self.br = tf.TransformBroadcaster()

        # Assume T is the given 4x4 transformation matrix
        self.T = np.ones((4, 4))  # Set to a 4x4 matrix

        # Set a timer to call the broadcast_transform function every 100 milliseconds
        self.timer = rospy.Timer(rospy.Duration(0.1), self.broadcast_transform)

        # Store the timestamp of the last sent transformation
        self.last_sent_time = None

    def broadcast_transform(self, event):
        try:
            # Extract the translation and rotation parts from the 4x4 matrix
            translation = self.T[0:3, 3]  # Translation part (x, y, z)
            rotation_matrix = self.T[0:3, 0:3]  # Rotation matrix part

            # Create a complete 4x4 matrix, including rotation and homogeneous coordinates
            full_matrix = np.eye(4)
            full_matrix[0:3, 0:3] = rotation_matrix
            full_matrix[0:3, 3] = translation

            # Create a quaternion to represent the rotation
            quaternion = tf.transformations.quaternion_from_matrix(full_matrix)

            # Get the current timestamp
            current_time = rospy.Time.now()

            # Check if the last sent timestamp and the current timestamp are the same
            if self.last_sent_time is None or current_time != self.last_sent_time:
                # Publish the transformation
                self.br.sendTransform(
                    (translation[0], translation[1], translation[2]),  # Translation part
                    (quaternion[0], quaternion[1], quaternion[2], quaternion[3]),  # Rotation part (quaternion)
                    current_time,  # Use the current timestamp
                    "cube",  # Child frame name
                    "world"   # Parent frame name
                )
                # Update the last sent timestamp
                self.last_sent_time = current_time
        except:
            pass

    def update(self, T):
        self.T = T
        
    def stop(self):
        # Stop the timer
        self.timer.shutdown()


## 2.2 Grasp Generation

In [13]:
from utils import *  # Assuming the create_grasp_mesh function is in utils.py
import numpy as np
import open3d as o3d

def generate_gripper_from_transform(T: np.ndarray):
    """
    Generates a robotic gripper mesh from a given 4x4 transformation matrix,
    with additional rotations around x and y axes.

    Args:
        T: 4x4 transformation matrix (numpy array).
        
    Returns:
        gripper_meshes: List of meshes representing the gripper.
    """
    # Extract the rotation matrix (3x3)
    rotation_matrix = T[:3, :3]

    # Extract the translation vector
    translation = T[:3, 3]

    # Set the gripper's center point position, usually the translation vector
    center_point = translation

    # Create a rotation matrix for -90 degrees around the x-axis
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(-np.pi/2), -np.sin(-np.pi/2)],
        [0, np.sin(-np.pi/2), np.cos(-np.pi/2)]
    ])

    # Create a rotation matrix for 90 degrees around the y-axis
    R_y = np.array([
        [np.cos(np.pi/2), 0, np.sin(np.pi/2)],
        [0, 1, 0],
        [-np.sin(np.pi/2), 0, np.cos(np.pi/2)]
    ])
    
    R_z = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])
    # Combine rotation matrices, first rotate around the x-axis, then around the y-axis
    combined_rotation = R_z @ rotation_matrix @ R_x 

    # Call create_grasp_mesh function to generate the gripper
    gripper_meshes = create_grasp_mesh(
        center_point=center_point, 
        rotation_matrix=combined_rotation,
        width=0.25
    )
    # Call create_grasp_mesh function to generate the gripper with a different rotation
    gripper_meshes_rotate = create_grasp_mesh(
        center_point=center_point, 
        rotation_matrix=rotation_matrix @ R_x,
        width=0.25
    )

    return gripper_meshes, gripper_meshes_rotate


In [14]:
# Test code: Pass in a 4x4 transformation matrix
T = np.array([
    [1, 0, 0, 0.1],  # Rotation matrix and translation
    [0, 1, 0, 0.2],
    [0, 0, 1, 0.3],
    [0, 0, 0, 1]
])

# Call the function to generate the gripper
gripper_meshes, _ = generate_gripper_from_transform(T)

# Visualize the generated gripper
o3d.visualization.draw_geometries(gripper_meshes)


error: XDG_RUNTIME_DIR not set in the environment.


In [15]:
def check_grasp_collision(
    grasp_meshes: Sequence[o3d.geometry.TriangleMesh],
    object_pcd: o3d.geometry.TriangleMesh,
    num_colisions: int = 10,
    tolerance: float = 0.00001
) -> bool:
    """
    Checks for collisions between a gripper grasp pose and target object
    using point cloud sampling.

    Args:
        grasp_meshes: List of mesh geometries representing the gripper components
        object_mesh: Triangle mesh of the target object
        num_collisions: Threshold on how many points to check
        tolerance: Distance threshold for considering a collision (in meters)

    Returns:
        bool: True if collision detected between gripper and object, False otherwise
    """
    # Combine gripper meshes
    combined_gripper = o3d.geometry.TriangleMesh()
    for mesh in grasp_meshes:
        combined_gripper += mesh  # Combine multiple gripper meshes

    # Sample points from both meshes
    num_points = 5000  # Sample 5000 points from both gripper and target object
    #######################TODO#######################
    # Uniformly sample point clouds from both the gripper and object meshes
    gripper_pcd = combined_gripper.sample_points_uniformly(number_of_points=num_points)
    gripper_points = np.asarray(gripper_pcd.points)  # Point coordinates of the gripper point cloud
    object_points = np.asarray(object_pcd.points)  # Point coordinates of the target object point cloud
    ##################################################
    
    # Build KDTree for object points
    is_collision = False
    #######################TODO#######################
    collision_count = 0
    # Build a KDTree for the target object point cloud
    object_kdtree = o3d.geometry.KDTreeFlann(object_pcd)
    for gripper_point in gripper_points:
        # For each gripper point, find the nearest point in the target object point cloud
        _, _, distances = object_kdtree.search_knn_vector_3d(gripper_point, 1)  # Find the nearest neighbor
        
        # If the distance to the nearest neighbor is less than the tolerance, consider it a collision
        if distances[0] <= tolerance:
            collision_count += 1
            
            # Exit early if enough collisions are detected
            if collision_count >= num_colisions:
                is_collision = True
                break
    #######################TODO#######################

    return is_collision


## 2.3 identify image of cubes

In [12]:
# from openai import OpenAI
# from io import BytesIO
# from PIL import Image
# import base64
# import json
# SYSTEM_PROMPT = """Please act as an image recognition agent. 
# You will be given a square face of a block, 
# which is projected from a point cloud. 
# Your task is to recognize the following:

# Determine if this is a block face.
# Each face contains only one letter, 
# one pattern (just detect whether it's a pattern, no need to identify the exact pattern), 
# or is blank (only wood texture). 
# Please detect whether it is a letter, 
# a pattern, or blank. 
# Each of these may be rotated. 
# Please analyze all possible rotations in a clockwise direction: 0°, 90°, 180°, and 270°.
# There might be a circular border around the face. 
# Please detect if this border exists. 
# It's confirmed that the color of the border matches the color of the letter or pattern.
# The expected output is a JSON in the following format:
# {
#     "check": true/false, 
#     "c": char/"pattern"/"blank", 
#     "color": "green"/"yellow"/"red"/"blue"/"None", 
#     "rotation": 0/90/180/270, 
#     "circle": true/false
# }
# """

# api_key=""
# client = OpenAI(api_key=api_key)

# def encode_image(image, quality=100):
#     if image.mode != 'RGB':
#         image = image.convert('RGB')  # Convert to RGB
#     buffered = BytesIO()
#     image.save(buffered, format="JPEG", quality=quality) 
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")

# def gpt4o_analysis(image_path, quality=50):
#     with Image.open(image_path) as img:
#         img_b64_str = encode_image(img, quality=quality)
#     img_type = "image/jpeg"
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": SYSTEM_PROMPT},
#                     {
#                         "type": "image_url",
#                         "image_url": {"url": f"data:{img_type};base64,{img_b64_str}"},
#                     },
#                 ],
#             }from openai import OpenAI
# from io import BytesIO
# from PIL import Image
# import base64
# import json
# SYSTEM_PROMPT = """Please act as an image recognition agent. 
# You will be given a square face of a block, 
# which is projected from a point cloud. 
# Your task is to recognize the following:

# Determine if this is a block face.
# Each face contains only one letter, 
# one pattern (just detect whether it's a pattern, no need to identify the exact pattern), 
# or is blank (only wood texture). 
# Please detect whether it is a letter, 
# a pattern, or blank. 
# Each of these may be rotated. 
# Please analyze all possible rotations in a clockwise direction: 0°, 90°, 180°, and 270°.
# There might be a circular border around the face. 
# Please detect if this border exists. 
# It's confirmed that the color of the border matches the color of the letter or pattern.
# The expected output is a JSON in the following format:
# {
#     "check": true/false, 
#     "c": char/"pattern"/"blank", 
#     "color": "green"/"yellow"/"red"/"blue"/"None", 
#     "rotation": 0/90/180/270, 
#     "circle": true/false
# }
# """

# api_key=""
# client = OpenAI(api_key=api_key)

# def encode_image(image, quality=100):
#     if image.mode != 'RGB':
#         image = image.convert('RGB')  # Convert to RGB
#     buffered = BytesIO()
#     image.save(buffered, format="JPEG", quality=quality) 
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")

# def gpt4o_analysis(image_path, quality=50):
#     with Image.open(image_path) as img:
#         img_b64_str = encode_image(img, quality=quality)
#     img_type = "image/jpeg"
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {
#                 "role": "user",
#                 "content": [
#                     {"type": "text", "text": SYSTEM_PROMPT},
#                     {
#                         "type": "image_url",
#                         "image_url": {"url": f"data:{img_type};base64,{img_b64_str}"},
#                     },
#                 ],
#             }
#         ],
#     )
#     return response.choices[0].message.content
#         ],
#     )
#     return response.choices[0].message.content

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
import matplotlib.pyplot as plt
import cv2
from PointCloud2Image import enlarge_points_as_cubes, max_downsample_image, pointcloud_to_top_view_image_color, interpolate_sparse_image, pointcloud_to_colored_image_with_filling, triangle_mesh_to_image

try:
    broadcaster
except NameError:
    broadcaster = TransformBroadcaster()           
    
cube_num = 10
# def pointcloud_process(point_cloud, slice_tolerance=0.005):
#     '''
#         Identify the cubes
#         Returns:
#             [
#                 json,
#                 T
#             ]
#     '''
#     orignal_point_cloud = copy.deepcopy(point_cloud)
#     T = []
#     remaining_pointcloud_count = 10000
#     countdown = 50
#     # Use open3d to visualize the point cloud
#     # o3d.visualization.draw_geometries([layer_point_cloud], window_name=f"Layer {layer} (Z range: {z_min:.4f} to {z_max:.4f})")
#     movecount = 0
#     while remaining_pointcloud_count > 50 and countdown > 0:
#         cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
#         cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
#         transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(point_cloud, cube_point_cloud)
#         remaining_pointcloud_count = len(remaining_pointcloud.points)
#         # return transform, remaining_pointcloud, deleted_pointcloud
#         if fitness > 0.01:
#             countdown = countdown - 1
#         print(fitness)
#         if check_transform_z_axis_alignment(transform) and fitness > 0.50 and np.array_equal(transform, align_transform_z_axis(transform)):
#         # if fitness > 0.70:
#             if movecount >= cube_num-1:
#                 break
#             print(movecount)
            

#             broadcaster.update(transform)
#             cube_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
#                 size=0.1,  # Coordinate axis size, can be adjusted as needed
#                 origin=[0, 0, 0]  # Origin of the coordinate axis
#             )
#             theta = np.radians(45)  # Convert angle to radians
#             transform_matrix_x_180 = np.array([
#                 [1, 0, 0, 0],
#                 [0, -1, 0, 0],
#                 [0, 0, -1, 0],
#                 [0, 0, 0, 1]
#             ])
#             transform_matrix_z_90 = np.array([
#                 [0, -1, 0, 0],
#                 [1, 0, 0, 0],
#                 [0, 0, 1, 0],
#                 [0, 0, 0, 1]
#             ])
#             graps_transform = transform @ transform_matrix_x_180
#             graps_transform_rotate = transform @ transform_matrix_x_180 @ transform_matrix_z_90
#                         # o3d.visualization.draw_geometries([coordinate_frame, remaining_pointcloud], window_name="remaining_pointcloud")
#             cube_point_cloud_transormed = copy.deepcopy(deleted_pointcloud)
#             cube_point_cloud_transormed = cube_point_cloud_transormed.transform(np.linalg.inv(graps_transform))
#             cube_point_cloud_transormed_cubes = enlarge_points_as_cubes(cube_point_cloud_transormed)

#             cube_top_image = triangle_mesh_to_image(cube_point_cloud_transormed_cubes, image_size=(100, 100))
#             cube_top_image = (cube_top_image / cube_top_image.max() * 255).astype(np.uint8)
#             grasp_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
#                 size=0.1,  # Coordinate axis size, can be adjusted as needed
#                 origin=[0, 0, 0]  # Origin of the coordinate axis
#             )
#             grasp_mesh, gripper_meshes_rotate = generate_gripper_from_transform(graps_transform)
#             # Apply transformation matrix to the coordinate frame
#             grasp_coordinate_frame.transform(graps_transform)
#             grasp_final_matrix = None
#             # o3d.visualization.draw_geometries(grasp_mesh+gripper_meshes_rotate+[orignal_point_cloud, coordinate_frame], window_name="remaining_pointcloud")
#             if check_grasp_collision(grasp_mesh, orignal_point_cloud):
#             # if check_grasp_collision(grasp_mesh, deleted_pointcloud):
#                 # If collision
#                 grasp_mesh = []
#             else:
#                 grasp_final_matrix = graps_transform
#             if check_grasp_collision(gripper_meshes_rotate, orignal_point_cloud):
#             # if check_grasp_collision(gripper_meshes_rotate, deleted_pointcloud):
#                 gripper_meshes_rotate = []
#             else:
#                 grasp_final_matrix = graps_transform_rotate
#             # o3d.visualization.draw_geometries(grasp_mesh+gripper_meshes_rotate+[grasp_coordinate_frame, deleted_pointcloud, coordinate_frame], window_name="deleted_pointcloud")
#             # o3d.visualization.draw_geometries(grasp_mesh+gripper_meshes_rotate+[grasp_coordinate_frame, remaining_pointcloud, coordinate_frame], window_name="deleted_pointcloud")
#             if grasp_final_matrix is not None: # Can move
#                 countdown = 50
#                 print(grasp_final_matrix)
#                 movecount += 1
#                 pick_rpy, pick_pos = matrix_to_rpy_and_translation(grasp_final_matrix)
#                 pick_pos_ = [a + b for a, b in zip(pick_pos, [0, 0, 0.10])]
#                 pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
#                 # pick_place.move(pick_pos_, pick_rpy)
#                 print(pick_pos_, pick_rpy)
#                 # cube_top_image = point_cloud_to_image(cube_point_cloud_transormed)
#                 plt.imsave(f"test_{movecount}.png", cube_top_image)
#                 plt.show()
#                 point_cloud = remaining_pointcloud
#                 # TODO: Identify the first face
            
#                 T.append(grasp_final_matrix)
#         else:
#             cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000)
#             cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
#     print(T)
#     return T

# def pointcloud_process(point_cloud, slice_tolerance=0.005, cube_num=cube_num):
#     '''
#     Identify the cubes in a point cloud and determine their transformation matrices for grasping.
#     Handles alignment of Z-axis in any orientation for proper grasping.
    
#     Args:
#         point_cloud: The input point cloud containing cubes to be identified
#         slice_tolerance: Tolerance for slicing the point cloud
#         cube_num: Maximum number of cubes to detect
        
#     Returns:
#         A list of transformation matrices for grasping the identified cubes
#     '''
#     original_point_cloud = copy.deepcopy(point_cloud)
#     T = []
#     remaining_pointcloud = point_cloud
#     remaining_pointcloud_count = len(point_cloud.points)
#     countdown = 50
#     movecount = 0
    
#     # Define standard transformation matrices for grasping
#     transform_matrix_x_180 = np.array([
#         [1, 0, 0, 0],
#         [0, -1, 0, 0],
#         [0, 0, -1, 0],
#         [0, 0, 0, 1]
#     ])
    
#     transform_matrix_z_90 = np.array([
#         [0, -1, 0, 0],
#         [1, 0, 0, 0],
#         [0, 0, 1, 0],
#         [0, 0, 0, 1]
#     ])
    
#     # Function to check if a cube's z-axis points approximately downward
#     def is_z_axis_pointing_down(transform):
#         z_axis = transform[:3, 2]  # Extract z-axis vector
#         return np.dot(z_axis, np.array([0, 0, -1])) > 0.8  # Check if close to pointing down
    
#     while remaining_pointcloud_count > 50 and countdown > 0 and movecount < cube_num:
#         # Sample points from the cube mesh model
#         cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
#         cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)
        
#         # Register the cube in the point cloud
#         transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(
#             remaining_pointcloud, cube_point_cloud
#         )
        
#         remaining_pointcloud_count = len(remaining_pointcloud.points)
        
#         # Decrement countdown if the fitness is above a minimum threshold
#         if fitness > 0.01:
#             countdown -= 1
            
#         print(f"Iteration {50-countdown}: Fitness = {fitness:.4f}, Remaining points: {remaining_pointcloud_count}")
        
#         # Check if the fitness is above the acceptable threshold
#         if fitness > 0.40:  # Lowered threshold slightly for more detections
#             print(f"Processing cube #{movecount+1}")
            
#             # Align the Z-axis to point downward regardless of original orientation
#             aligned_transform = align_transform_z_axis_any_orientation(transform)
            
#             # Update the transform broadcaster for visualization
#             if 'broadcaster' in globals():
#                 broadcaster.update(aligned_transform)
            
#             # Create grasp transforms
#             grasp_transform = aligned_transform @ transform_matrix_x_180
#             grasp_transform_rotate = aligned_transform @ transform_matrix_x_180 @ transform_matrix_z_90
            
#             # Transform the detected cube point cloud for visualization
#             cube_point_cloud_transformed = copy.deepcopy(deleted_pointcloud)
#             cube_point_cloud_transformed = cube_point_cloud_transformed.transform(np.linalg.inv(grasp_transform))
            
#             # For generating a visual representation as a top-down view
#             try:
#                 cube_point_cloud_transformed_cubes = enlarge_points_as_cubes(cube_point_cloud_transformed)
#                 cube_top_image = triangle_mesh_to_image(cube_point_cloud_transformed_cubes, image_size=(100, 100))
#                 cube_top_image = (cube_top_image / cube_top_image.max() * 255).astype(np.uint8)
#                 plt.imsave(f"cube_{movecount+1}_top_view.png", cube_top_image)
#             except Exception as e:
#                 print(f"Warning: Could not generate top view image: {e}")
            
#             # Generate gripper meshes for collision checking
#             grasp_mesh, gripper_meshes_rotate = generate_gripper_from_transform(grasp_transform)
            
#             # Find a valid grasp without collisions
#             grasp_final_matrix = None
            
#             # Check first grasping orientation
#             if not check_grasp_collision(grasp_mesh, original_point_cloud):
#                 grasp_final_matrix = grasp_transform
#                 print("Using standard grasp orientation")
#             # Check alternative grasping orientation (rotated by 90° around Z)
#             elif not check_grasp_collision(gripper_meshes_rotate, original_point_cloud):
#                 grasp_final_matrix = grasp_transform_rotate
#                 print("Using 90° rotated grasp orientation")
#             else:
#                 print("Both grasp orientations have collisions, skipping this cube")
                
#             # If a valid grasp pose was found
#             if grasp_final_matrix is not None:
#                 # Reset countdown
#                 countdown = 50
                
#                 # Calculate RPY and position for robot movement
#                 pick_rpy, pick_pos = matrix_to_rpy_and_translation(grasp_final_matrix)
#                 approach_offset = [0, 0, 0.10]  # 10cm above the cube for approach
#                 pick_pos_approach = [a + b for a, b in zip(pick_pos, approach_offset)]
                
#                 print(f"Cube #{movecount+1} grasp position: {pick_pos}")
#                 print(f"Approach position: {pick_pos_approach}")
#                 print(f"RPY orientation: {pick_rpy}")
                
#                 # Add the transform to our results
#                 T.append(grasp_final_matrix)
#                 movecount += 1
                
#                 # Optional visualization of the detected cube
#                 # o3d.visualization.draw_geometries([deleted_pointcloud], window_name=f"Detected Cube #{movecount}")
                
#                 # Optional visualization of the grasp
#                 # vis_geometries = grasp_mesh + [create_coordinate_frame(0.1)]
#                 # o3d.visualization.draw_geometries(vis_geometries, window_name=f"Grasp Visualization #{movecount}")
#         else:
#             # If fitness is too low, don't change the countdown
#             pass
    
#     print(f"Found {len(T)} cubes from {50-countdown} iterations")
    
#     # Print the final transforms
#     for i, transform in enumerate(T):
#         print(f"Cube #{i+1} Transform:")
#         print(transform)
        
#     return T
def pointcloud_process(point_cloud, slice_tolerance=0.005, cube_num=4):
    '''
    Identify cubes within a point cloud and determine transformation matrices for grasping.
    Uses improved Z-axis alignment method that preserves XY plane consistency to reduce alignment errors.
    
    Args:
        point_cloud: The input point cloud containing cubes to be detected
        slice_tolerance: Tolerance value for slicing the point cloud
        cube_num: Maximum number of cubes to detect
        
    Returns:
        A list of transformation matrices corresponding to the detected graspable cubes
    '''
    original_point_cloud = copy.deepcopy(point_cloud)
    T = []
    remaining_pointcloud = point_cloud
    remaining_pointcloud_count = len(point_cloud.points)
    countdown = 50
    movecount = 0

    # Define standard transformation matrices for grasping orientation
    transform_matrix_x_180 = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ])

    transform_matrix_z_90 = np.array([
        [0, -1, 0, 0],
        [1,  0, 0, 0],
        [0,  0, 1, 0],
        [0,  0, 0, 1]
    ])

    # Iterate to find and extract cubes
    while remaining_pointcloud_count > 50 and countdown > 0 and movecount < cube_num:
        # Sample points from the cube mesh model
        cube_point_cloud = cube_mesh.sample_points_uniformly(number_of_points=50000) 
        cube_point_cloud = filter_point_cloud_by_depth(cube_point_cloud, depth_threshold=-0.01)

        # Register cube with the current remaining scene point cloud
        transform, remaining_pointcloud, deleted_pointcloud, fitness = register_and_filter(
            remaining_pointcloud, cube_point_cloud
        )

        remaining_pointcloud_count = len(remaining_pointcloud.points)

        # Reduce countdown if fitness exceeds minimum threshold
        if fitness > 0.01:
            countdown -= 1

        print(f"Iteration {50 - countdown}: Fitness = {fitness:.4f}, Remaining points: {remaining_pointcloud_count}")

        # Proceed only if the match is good enough
        if fitness > 0.40:  # Slightly lowered threshold to detect more cubes
            print(f"Processing cube #{movecount+1}")

            # Align the Z-axis downward while preserving XY orientation
            aligned_transform = align_transform_z_axis_preserving_xy(transform)

            # Update transform broadcaster (for visualization in RViz, etc.)
            if 'broadcaster' in globals():
                broadcaster.update(aligned_transform)

            # Create grasping transformations
            grasp_transform = aligned_transform @ transform_matrix_x_180
            grasp_transform_rotate = aligned_transform @ transform_matrix_x_180 @ transform_matrix_z_90

            # Transform the detected cube point cloud (for visualization or checking)
            cube_point_cloud_transformed = copy.deepcopy(deleted_pointcloud)
            cube_point_cloud_transformed = cube_point_cloud_transformed.transform(np.linalg.inv(grasp_transform))

            # Try generating a top-view image of the detected cube
            try:
                cube_point_cloud_transformed_cubes = enlarge_points_as_cubes(cube_point_cloud_transformed)
                cube_top_image = triangle_mesh_to_image(cube_point_cloud_transformed_cubes, image_size=(100, 100))
                cube_top_image = (cube_top_image / cube_top_image.max() * 255).astype(np.uint8)
                plt.imsave(f"cube_{movecount+1}_top_view.png", cube_top_image)
            except Exception as e:
                print(f"Warning: Could not generate top view image: {e}")

            # Generate gripper meshes for collision checking
            grasp_mesh, gripper_meshes_rotate = generate_gripper_from_transform(grasp_transform)

            # Try both grasping orientations and choose one that doesn't collide
            grasp_final_matrix = None

            # Try standard grasp orientation
            if not check_grasp_collision(grasp_mesh, original_point_cloud):
                grasp_final_matrix = grasp_transform
                print("Using standard grasp orientation")
            # Try alternative 90° rotated grasp
            elif not check_grasp_collision(gripper_meshes_rotate, original_point_cloud):
                grasp_final_matrix = grasp_transform_rotate
                print("Using 90° rotated grasp orientation")
            else:
                print("Both grasp orientations have collisions, skipping this cube")

            # If a valid grasp was found, store it
            if grasp_final_matrix is not None:
                countdown = 50  # Reset countdown

                # Calculate RPY and position for motion planning
                pick_rpy, pick_pos = matrix_to_rpy_and_translation(grasp_final_matrix)
                approach_offset = [0, 0, 0.10]  # 10 cm above cube
                pick_pos_approach = [a + b for a, b in zip(pick_pos, approach_offset)]

                print(f"Cube #{movecount+1} grasp position: {pick_pos}")
                print(f"Approach position: {pick_pos_approach}")
                print(f"RPY orientation: {pick_rpy}")

                # Append the transform to the result list
                T.append(grasp_final_matrix)
                movecount += 1

                # Optional: show the grasp transform matrix
                print("Grasp transform matrix:")
                print(grasp_final_matrix)

                # Optional: visualize grasp
                # vis_geometries = grasp_mesh + [create_coordinate_frame(0.1)]
                # o3d.visualization.draw_geometries(vis_geometries, window_name=f"Grasp Visualization #{movecount}")

    print(f"Found {len(T)} cubes in {50 - countdown} iterations")

    # Print final grasp transforms
    for i, transform in enumerate(T):
        print(f"Cube #{i+1} Transform:")
        print(transform)

    return T

            
Ts = pointcloud_process(filtered_point_cloud)
broadcaster.stop()


迭代 1: 拟合度 = 0.8412, 剩余点数: 19106
处理立方体 #1
Z-axis alignment quality: 1.000000 (closer to 1 is better)
XY plane preservation quality: 0.998749 (closer to 1 is better)
使用标准抓取方向
立方体 #1 抓取位置: [0.5289837279170989, -0.010098020185695014, 0.024541929567309734]
接近位置: [0.5289837279170989, -0.010098020185695014, 0.12454192956730974]
RPY方向: [0.0, 0.0, 0.33864135392116157]
抓取变换矩阵:
[[ 0.94320689 -0.33220591  0.          0.52898373]
 [ 0.33220591  0.94320689  0.         -0.01009802]
 [ 0.          0.          1.          0.02454193]
 [ 0.          0.          0.          1.        ]]
迭代 1: 拟合度 = 0.8199, 剩余点数: 16676
处理立方体 #2
Z-axis alignment quality: 1.000000 (closer to 1 is better)
XY plane preservation quality: 0.998788 (closer to 1 is better)
使用标准抓取方向
立方体 #2 抓取位置: [0.4970921171222455, 0.30115411737378356, 0.02445701548324259]
接近位置: [0.4970921171222455, 0.30115411737378356, 0.12445701548324259]
RPY方向: [0.0, 0.0, 0.5710685147446949]
抓取变换矩阵:
[[ 0.84132389 -0.54053132  0.          0.49709212]
 [ 0.54053

In [39]:
Ts.__len__()

8

In [18]:
import numpy as np
import open3d as o3d
import copy

def generate_pascal_triangle_transforms(t, T):
    """
    Generate a list of transformation matrices for a Pascal triangle arrangement.
    Each cube's center position is used as the translation part of the transform matrix.
    """
    level = 1
    total = 1
    while total < t:
        level += 1
        total += level

    transforms = []
    cube_size = 0.045  # 4.5 cm
    spacing_xy = cube_size * 1.25  # Increase spacing to 1.25 times the cube size
    spacing_z = cube_size * 1.05   # Use the same spacing in the vertical direction

    current_pos = 0
    for row in range(level-1, -1, -1):
        for col in range(row + 1):
            if current_pos >= t:
                break

            center_x = 0
            center_y = (col - row/2) * spacing_xy
            center_z = (level - 1 - row) * spacing_z

            # Create local transformation matrix
            local_transform = np.eye(4)
            local_transform[:3, 3] = [center_x, center_y, center_z]

            # Combine local transformation with T transformation
            transform = np.dot(T, local_transform)
            transforms.append(transform)
            current_pos += 1

        if current_pos >= t:
            break

    return transforms

def create_coordinate_frame(size=0.1, transform=None):
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    if transform is not None:
        frame.transform(transform)
    return frame

def visualize_pascal_triangle(transforms, T):
    # Create a cube centered at the origin
    cube = o3d.geometry.TriangleMesh.create_box(
        width=0.045,
        height=0.045, 
        depth=0.045
    )
    # Move the cube to be centered at the origin
    cube.translate([-0.045/2, -0.045/2, -0.045/2])
    cube.compute_vertex_normals()

    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Add world coordinate frame
    world_frame = create_coordinate_frame(size=0.2)
    vis.add_geometry(world_frame)

    # Add T coordinate frame
    t_frame = create_coordinate_frame(size=0.2, transform=T)
    vis.add_geometry(t_frame)

    # Add all cubes and their local coordinate frames
    for transform in transforms:
        # Add cube
        cube_copy = copy.deepcopy(cube)
        cube_copy.transform(transform)
        vis.add_geometry(cube_copy)
        
        # Add local coordinate frame
        local_frame = create_coordinate_frame(size=0.05, transform=transform)
        vis.add_geometry(local_frame)

    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])

    ctr = vis.get_view_control()
    ctr.set_zoom(0.2)  # Adjust zoom to fit larger spacing
    ctr.set_front([-0.8, -0.5, 0.5])
    ctr.set_lookat([0, 0, 0])
    ctr.set_up([0, 0, 1])

    vis.run()
    vis.destroy_window()


# 3. Motion Planning, movement, and grasp

In [19]:
from utils import generate_pascal_triangle_transforms
import numpy as np

In [20]:
# t = Ts.__len__()
t = 2
# 创建T矩阵（示例：绕Z轴旋转45度并平移）
T = np.eye(4)
theta = 0
T[:3, :3] = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
])
T[:3, 3] = [0.3, 0.00, 0]

aim_transforms = generate_pascal_triangle_transforms(t, T)
# visualize_pascal_triangle(aim_transforms, T)

In [21]:
aim_transforms

[array([[ 1.      ,  0.      ,  0.      ,  0.3     ],
        [ 0.      ,  1.      ,  0.      , -0.028125],
        [ 0.      ,  0.      ,  1.      ,  0.      ],
        [ 0.      ,  0.      ,  0.      ,  1.      ]]),
 array([[1.      , 0.      , 0.      , 0.3     ],
        [0.      , 1.      , 0.      , 0.028125],
        [0.      , 0.      , 1.      , 0.      ],
        [0.      , 0.      , 0.      , 1.      ]])]

In [40]:
Ts

[array([[ 0.94320689, -0.33220591,  0.        ,  0.52898373],
        [ 0.33220591,  0.94320689,  0.        , -0.01009802],
        [ 0.        ,  0.        ,  1.        ,  0.02454193],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.84132389, -0.54053132,  0.        ,  0.49709212],
        [ 0.54053132,  0.84132389,  0.        ,  0.30115412],
        [ 0.        ,  0.        ,  1.        ,  0.02445702],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.80030483, -0.59959334,  0.        ,  0.39361411],
        [ 0.59959334,  0.80030483,  0.        ,  0.11186747],
        [ 0.        ,  0.        ,  1.        ,  0.02475005],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.88688204, -0.46199593,  0.        ,  0.5297414 ],
        [ 0.46199593,  0.88688204,  0.        , -0.11133846],
        [ 0.        ,  0.        ,  1.        ,  0.0246574 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),


In [ ]:
# # debug:
# import numpy as np

# Ts = [np.array([[-0.30936624, -0.95090032,  0.00900657,  0.46123814],
#         [-0.95038183,  0.30949514,  0.03141855, -0.07133785],
#         [-0.0326634 ,  0.00116016, -0.99946574,  0.03173297],
#         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
#  np.array([[ 0.9979552 , -0.0391174 , -0.05054952,  0.61442553],
#         [-0.04106496, -0.99843052, -0.03808115,  0.10012986],
#         [-0.04898055,  0.0400791 , -0.99799528,  0.01518867],
#         [ 0.        ,  0.        ,  0.        ,  1.        ]]),
#  # np.array([[ 0.97671909, -0.17776618,  0.12007919,  0.4728227 ],
#  #        [-0.19286849, -0.97274425,  0.12872592,  0.08761278],
#  #        [ 0.09392323, -0.14888855, -0.98438337,  0.02699751],
#  #        [ 0.        ,  0.        ,  0.        ,  1.        ]])
#      ]


In [ ]:
# demostrate the movement to the head of identified cubes
# for T in Ts:
#     pick_rpy, pick_pos = matrix_to_rpy_and_translation(T)
#     pick_pos_ = [a + b for a, b in zip(pick_pos, [0.04, 0.00, 0.10])]
#     pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
#     print(pick_rpy, pick_pos)
#     pick_place.move(pick_pos_, pick_rpy)
# Modified movement demonstration code
def demonstrate_movement_to_cubes(Ts, pick_place, offset=[0.04, 0.00, 0.10]):
    for i, T in enumerate(Ts):
        try:
            pick_rpy, pick_pos = matrix_to_rpy_and_translation(T)
            pick_pos_approach = [a + b for a, b in zip(pick_pos, offset)]
            
            try:
                pick_place.move(pick_pos_approach, pick_rpy)
                print(f"Successfully moved to approach position for cube #{i+1}")
            except ValueError as e:
                if "too many values to unpack" in str(e):
                    print(f"Note: Handled unpacking error in move operation for cube #{i+1}")
                else:
                    raise
        except Exception as e:
            print(f"Error processing cube #{i+1}: {e}")

# Use the modified function
demonstrate_movement_to_cubes(Ts, pick_place)


[0.0, 3.141592653589793, 0.33864135392116157]
Successfully moved to approach position for cube #1
[0.0, 3.141592653589793, 0.5710685147446949]
Successfully moved to approach position for cube #2
[0.0, 3.141592653589793, 0.6429928816452974]
Successfully moved to approach position for cube #3
[0.0, 3.141592653589793, 0.48024438614592585]
Successfully moved to approach position for cube #4
[0.0, 3.141592653589793, -0.3036735721124885]
Successfully moved to approach position for cube #5
[0.0, 3.141592653589793, -2.3186944946043067]
Successfully moved to approach position for cube #6
[0.0, 3.141592653589793, -0.3851767417558013]
Successfully moved to approach position for cube #7
[0.0, 3.141592653589793, -0.4246896206614637]
Successfully moved to approach position for cube #8


[ERROR] [1742599891.027330, 3369.298000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.032058, 3369.302000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.039970, 3369.303000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.047647, 3369.303000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.055904, 3369.303000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.087132, 3369.343000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.096378, 3369.352000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599891.102864, 3369.359000]: Motion planning failed. No valid plan generated.


In [ ]:
# move the minipulator to the aim position
for T in aim_transforms:
    transform_matrix_x_180 = np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ])
    transform_matrix_z_90 = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    pick_rpy, pick_pos = matrix_to_rpy_and_translation(T@transform_matrix_x_180@transform_matrix_z_90)
    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.0, 0, 0.13])]
    pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
    pick_place.move(pick_pos_, pick_rpy)


[ERROR] [1742598238.893879, 1898.446000]: Motion planning failed. No valid plan generated.
[ERROR] [1742598238.903600, 1898.446000]: Motion planning failed. No valid plan generated.


([3.141592653589793, 0.0, -1.5707963267948966],
 [0.3, 0.028124999999999997, 0.0])

In [32]:
# pick and place the first cube 
transform_matrix_x_180 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])
transform_matrix_z_90 = np.array([
    [0, -1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

pick_rpy, pick_pos = matrix_to_rpy_and_translation(Ts[0])
pick_pos_ = [a + b for a, b in zip(pick_pos, [0.04, 0, 0.00])]
pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]
    
place_rpy, place_pos = matrix_to_rpy_and_translation(aim_transforms[0]@transform_matrix_x_180@transform_matrix_z_90)
place_pos_ = [a + b for a, b in zip(place_pos, [0.0, 0, 0.03])]

pick_place.pick_and_place(
    pick_pos=pick_pos_,
    pick_rpy=pick_rpy,
    place_pos=place_pos_,
    place_rpy=place_rpy
)

[INFO] [1742598142.215533, 1812.803000]: Sending open goal: width: 0.08
speed: 0.1
[INFO] [1742598142.395808, 1812.944000]: Gripper opened successfully.


[ERROR] [1742598143.403141, 1813.823000]: Motion planning failed. No valid plan generated.
[ERROR] [1742598143.611586, 1814.044000]: Motion planning failed. No valid plan generated.


[INFO] [1742598143.919487, 1814.316000]: Sending grasp goal: width: 0.04
epsilon: 
  inner: 0.02
  outer: 0.02
speed: 0.1
force: 1.0


[WARN] [1742598145.036395, 1815.319000]: Grasp failed.
[ERROR] [1742598145.345063, 1815.584000]: Motion planning failed. No valid plan generated.
[ERROR] [1742598145.550580, 1815.748000]: Motion planning failed. No valid plan generated.
[ERROR] [1742598146.056421, 1816.173000]: Motion planning failed. No valid plan generated.


[INFO] [1742598146.569202, 1816.620000]: Sending open goal: width: 0.07
speed: 0.1
[INFO] [1742598147.517664, 1817.455000]: Gripper opened successfully.


[ERROR] [1742598148.041935, 1817.949000]: Motion planning failed. No valid plan generated.


In [ ]:
# pick and place following cubes 
for i in range(1, Ts.__len__()-1):
    ptransform_matrix_x_180 = np.array([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
    ])
    transform_matrix_z_90 = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    pick_rpy, pick_pos = matrix_to_rpy_and_translation(Ts[i])
    pick_pos_ = [a + b for a, b in zip(pick_pos, [0.0, 0, 0.008])]
    pick_rpy = [a + b for a, b in zip(pick_rpy, [0, 0, 0])]

    place_rpy, place_pos = matrix_to_rpy_and_translation(aim_transforms[i]@transform_matrix_x_180@transform_matrix_z_90)
    place_pos_ = [a + b for a, b in zip(place_pos, [0.0, 0, 0.03])]

    pick_place.pick_and_place(
        pick_pos=pick_pos_,
        pick_rpy=pick_rpy,
        place_pos=place_pos_,
        place_rpy=place_rpy
    )

[INFO] [1742599801.925197, 3291.629000]: Sending open goal: width: 0.08
speed: 0.1
[INFO] [1742599801.974230, 3291.652000]: Gripper opened successfully.


[ERROR] [1742599802.983268, 3292.550000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599803.191008, 3292.719000]: Motion planning failed. No valid plan generated.


[INFO] [1742599803.494745, 3293.000000]: Sending grasp goal: width: 0.04
epsilon: 
  inner: 0.02
  outer: 0.02
speed: 0.1
force: 1.0


[WARN] [1742599804.662736, 3293.996000]: Grasp failed.
[ERROR] [1742599804.965281, 3294.274000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599805.169579, 3294.412000]: Motion planning failed. No valid plan generated.
[ERROR] [1742599805.690936, 3294.873000]: Motion planning failed. No valid plan generated.


[INFO] [1742599806.209010, 3295.339000]: Sending open goal: width: 0.07
speed: 0.1
[INFO] [1742599807.217041, 3296.211000]: Gripper opened successfully.


[ERROR] [1742599807.723500, 3296.678000]: Motion planning failed. No valid plan generated.


IndexError: list index out of range

[[ 0.94320689 -0.33220591  0.          0.52898373]
 [ 0.33220591  0.94320689  0.         -0.01009802]
 [ 0.          0.          1.          0.02454193]
 [ 0.          0.          0.          1.        ]]
([0.0, 0.0, 0.33864135392116157], [0.5289837279170989, -0.010098020185695014, 0.024541929567309734])
[[ 0.84132389 -0.54053132  0.          0.49709212]
 [ 0.54053132  0.84132389  0.          0.30115412]
 [ 0.          0.          1.          0.02445702]
 [ 0.          0.          0.          1.        ]]
([0.0, 0.0, 0.5710685147446949], [0.4970921171222455, 0.30115411737378356, 0.02445701548324259])
[[ 0.80030483 -0.59959334  0.          0.39361411]
 [ 0.59959334  0.80030483  0.          0.11186747]
 [ 0.          0.          1.          0.02475005]
 [ 0.          0.          0.          1.        ]]
([0.0, 0.0, 0.6429928816452974], [0.3936141102195277, 0.1118674722427118, 0.024750046723177994])
[[ 0.88688204 -0.46199593  0.          0.5297414 ]
 [ 0.46199593  0.88688204  0.         -

[array([[ 0.94320689, -0.33220591,  0.        ,  0.52898373],
        [ 0.33220591,  0.94320689,  0.        , -0.01009802],
        [ 0.        ,  0.        ,  1.        ,  0.02454193],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.84132389, -0.54053132,  0.        ,  0.49709212],
        [ 0.54053132,  0.84132389,  0.        ,  0.30115412],
        [ 0.        ,  0.        ,  1.        ,  0.02445702],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.80030483, -0.59959334,  0.        ,  0.39361411],
        [ 0.59959334,  0.80030483,  0.        ,  0.11186747],
        [ 0.        ,  0.        ,  1.        ,  0.02475005],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.88688204, -0.46199593,  0.        ,  0.5297414 ],
        [ 0.46199593,  0.88688204,  0.        , -0.11133846],
        [ 0.        ,  0.        ,  1.        ,  0.0246574 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
